In [1]:
import pandas as pd
import numpy as np
import re

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('case_study_data_ver1.csv', sep = ';')

In [3]:
df = df.dropna() # getting rid of all NULLs

In [4]:
df['QTY'] = df['QTY'].astype(str)
outtakes = df[df.QTY.str.contains('[a-zA-Z]|\.\d\d$|\.\d\d\d\d$')]
df = df.drop(outtakes.index)
df['QTY'] = df['QTY'].apply(lambda x: re.sub('\.','',re.sub('\..$','',x))).astype(float)
df = df.drop([key for key in list(df.keys()) if df[key].sum()==0], axis=1)

In [5]:
df = df[(df.STORE_ID==3)&(df.ITEM_ID==4)]

In [17]:
# add additional features last 30 days
# for val in df.QTY.values[:3]:
#     print(val)
# for i in range(3):
#     print(df.QTY.values[i],df.QTY.values[i-1])

qty_prev_1d = []
for i in range(len(df)):
    qty_prev_1d.append(df.QTY.values[i-1])

In [32]:
print(qty_prev_1d[:2])
print(qty_prev_1d[len(qty_prev_1d)-1], '\n')

print(df.QTY.head()[:2])
print(df.QTY.tail()[-2:])

# we should then drop the first row, since no 'real' previous data is available

[10.0, 20.0]
48.0 

97     20.0
681     7.0
Name: QTY, dtype: float64
955570    48.0
979799    10.0
Name: QTY, dtype: float64


In [27]:
print(df.QTY.head())
print(df.QTY.tail())


97      20.0
681      7.0
1192     4.0
1790     8.0
2431     7.0
Name: QTY, dtype: float64
952091    12.0
953244     7.0
954394    21.0
955570    48.0
979799    10.0
Name: QTY, dtype: float64


In [8]:
# Exclude: 
# - ID's, 
# - what we can't know at prediction time 
features = [col for col in list(df.keys()) if (col not in list(df.keys())[:3]) & \
                                              (col not in ['QTY', 'CUSTOMER'])]

In [9]:
# Split training & new data
df_validation = df[df.DATE_ID >= df.DATE_ID.max()-7]
df_training = df.drop(df_validation.index)